# Lab 4: PolyHAR CNN

## Imports

In [10]:
import copy
from pathlib import Path
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Input, Conv1D, MaxPool1D, Flatten, Dense, Activation
from keras.utils.np_utils import to_categorical

## Load PolyHAR dataset (EllcieHAR format)

In [11]:
x_full = []
y_full = []
with open('polyhar.csv') as f:
    next(f) # Skip header
    for l in f:
        d = l.split(';')
        x_full.append([float(d[1]), float(d[2]), float(d[3])]) # Store 3-axis accelerometer adata
        y_full.append(1 if 'Positive' in d[4] else 0) # Store positive labels

x_full = np.array(x_full)
y_full = np.array(y_full)

## Windowing

In [12]:
SIZE = 32
CLASSES = 2
windowscount = np.ceil(x_full.shape[0]/SIZE).astype(int)
x_full = np.resize(x_full, (windowscount, SIZE, x_full.shape[-1]))
y_full = np.resize(y_full, (windowscount, SIZE))
y_full = np.array([np.bincount(w).argmax() for w in y_full]) # Select label with highest number of occurence for each window
y_full = to_categorical(y_full, num_classes=CLASSES) # Convert back to one-hot encoding

## Train/test split

In [13]:
RATIO = 0.2 # 20% test, 80% train
n = int(len(x_full) * RATIO)

# Randomize windows
p = np.random.permutation(len(x_full))
x_full = x_full[p]
y_full = y_full[p]

x_test = x_full[-n:]
y_test = y_full[-n:]

x_train = x_full[:-len(x_test)]
y_train = y_full[:-len(y_test)]

## Export pre-processed dataset

In [14]:
np.savetxt('x_train_polyhar.csv', x_train.reshape((x_train.shape[0], -1)), delimiter=',', fmt='%s')
np.savetxt('y_train_polyhar.csv', y_train, delimiter=',', fmt='%s')
np.savetxt('x_test_polyhar.csv', x_test.reshape((x_test.shape[0], -1)), delimiter=',', fmt='%s')
np.savetxt('y_test_polyhar.csv', y_test, delimiter=',', fmt='%s')

## Build model

In [15]:
model = Sequential()
model.add(Input(shape=(SIZE, 3)))
model.add(Conv1D(filters=2, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(units=CLASSES))
model.add(Activation('softmax')) # SoftMax activation needs to be separate from Dense to remove it later on
# EXPLORE Learning Rate
opt = tf.keras.optimizers.Adam(learning_rate=10e-5)
model.summary()
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_1 (Conv1D)           (None, 30, 2)             20        
                                                                 
 flatten_1 (Flatten)         (None, 60)                0         
                                                                 
 dense_1 (Dense)             (None, 2)                 122       
                                                                 
 activation_1 (Activation)   (None, 2)                 0         
                                                                 
Total params: 142
Trainable params: 142
Non-trainable params: 0
_________________________________________________________________


## Train model

In [16]:
model.fit(x_train, y_train, epochs=3, validation_data=(x_test, y_test))

Epoch 1/3
8/8 [==============================] - 3s 37ms/step - loss: 0.6760 - categorical_accuracy: 0.8922 - val_loss: 0.6639 - val_categorical_accuracy: 0.8966
Epoch 2/3
8/8 [==============================] - 0s 9ms/step - loss: 0.6749 - categorical_accuracy: 0.8836 - val_loss: 0.6624 - val_categorical_accuracy: 0.8966
Epoch 3/3
8/8 [==============================] - 0s 9ms/step - loss: 0.6737 - categorical_accuracy: 0.8836 - val_loss: 0.6609 - val_categorical_accuracy: 0.8966


## Evaluate model on test dataset

In [17]:
model.evaluate(x_test, y_test, verbose=2)
pred_test = model.predict(x_test)
print(tf.math.confusion_matrix(y_test.argmax(axis=1), pred_test.argmax(axis=1)))

2/2 - 0s - loss: 0.6609 - categorical_accuracy: 0.8966 - 31ms/epoch - 15ms/step
2/2 [==============================] - 0s 5ms/step
tf.Tensor(
[[41  0]
 [ 6 11]], shape=(2, 2), dtype=int32)


## Save trained model

In [18]:
model.save('lab4_polyhar.h5')

## Remove SoftMax layer

In [19]:
model = tf.keras.Model(model.input, model.layers[-2].output, name=model.name)

## Install MicroAI for C inference code generation (kerascnn2c module)

In [20]:
!pip install https://bitbucket.org/edge-team-leat/microai_public/get/6adfbcb347d3.zip#subdirectory=third_party/kerascnn2c_fixed
import kerascnn2c

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 3.7 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for kerascnn2c: filename=kerascnn2c-1.0.0-py3-none-any.whl size=21339 sha256=377551b70414d3e8d6830e664f70555baf5451589adc49a0f63b446459e85e2e
  Stored in directory: /tmp/pip-ephem-wheel-cache-ydfsihk6/wheels/29/df/9b/d62a64e871a29555dc13bc0c189d46297cdf80a3332230aaa1
Successfully built kerascnn2c


## Generate C code for the trained model with 16-bit fixed-point representation

In [21]:
res = kerascnn2c.Converter(output_path=Path('polyhar_output_fixed'),
                           fixed_point=9, # Number of bits for the fractional part, Q7.9 format
                           number_type='int16_t', # Data type for weights/activations (16 bits quantization)
                           long_number_type='int32_t', # Data type for intermediate results
                           number_min=-(2**15), # Minimum value for 16-bit signed integers
                           number_max=(2**15)-1 # Maximum value for 16-bit signed integers
                          ).convert_model(copy.deepcopy(model))
with open('polyhar_model_fixed.h', 'w') as f:
    f.write(res)

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......conv1d
.........vars
............0
............1
......dense
.........vars
............0
............1
......flatten
.........vars
......input_layer
.........vars
...vars
Keras model archive saving:
File Name                                             Modified             Size
metadata.json                                  2023-03-31 14:33:13           64
config.json                                    2023-03-31 14:33:13         1749
variables.h5                                   2023-03-31 14:33:13        13600
Keras model archive loading:
File Name                                             Modified             Size
metadata.json                                  2023-03-31 14:33:12           64
config.json                                    2023-03-31 14:33:12         1749
variables.h5                                   2023-03-31 14:33:12        13600
Keras weights file (<HDF5 file "variables.h5" (mod

## Compile the 16-bit fixed-point C code for x86 and evaluate

In [26]:
!g++ -Wall -Wextra -pedantic -Ofast -o polyhar_fixed -Ipolyhar_output_fixed/ polyhar_output_fixed/model.c main.cpp 
!./polyhar_fixed x_test_polyhar.csv y_test_polyhar.csv

polyhar_output_fixed/model.c: In function ‘void cnn(const number_t (*)[32], number_t*)’:
polyhar_output_fixed/model.c:55:18: warning: left operand of comma operator has no effect []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-value-Wunused-value]8;;]
   55 |     activations1.conv1d_1_output,
      |     ~~~~~~~~~~~~~^~~~~~~~~~~~~~~
Testing accuracy: 0.896552
